In [5]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import pickle

from lib.rmodel import Dataset, BertClassifier, train, predict, evaluate

In [2]:
datapath = f'../while_waiting_for_store.csv'
df = pd.read_csv(datapath)
df=df[df.columns[2:]]
df.head()

,title,selftext,full_link,num_comments,score,retrieved_on,relevance
0,Best papers,TLDR: What papers in your sub field do you thi...,https://www.reddit.com/r/datascience/comments/...,0,1,1650307487,-1
1,I Need Help With My Honors Chemistry Project D...,# Hi so in my Honors Chem Project I measured t...,https://www.reddit.com/r/datascience/comments/...,0,1,1650310497,-1
2,Ubiquant Market Prediction EDA - Complete Sour...,NaN,https://www.reddit.com/r/datascience/comments/...,0,1,1650310673,-1
3,Hello Everyone! Im usuing the ARIMA Model to d...,NaN,https://www.reddit.com/r/datascience/comments/...,0,1,1650312837,-1
4,Can an Economics graduate be a Data Scientist?,The previous post was deleted due to insuffici...,https://www.reddit.com/r/datascience/comments/...,0,1,1650313913,-1


In [3]:
df = df.fillna(value=str(''))
df['text'] = df['title'] + '  ' + df['selftext']
df['text'] = df['text'].fillna(value='').astype(str)
df_clean = df[df.relevance!=-1]


In [6]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df_clean.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_clean)), int(.9*len(df_clean))])

print(len(df_train),len(df_val), len(df_test))

66 8 9


In [7]:
EPOCHS = 2
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████| 33/33 [00:48<00:00,  1.47s/it]


Epochs: 1 | Train Loss:  0.369 | Train Accuracy:  0.606 | Val Loss:  0.390 | Val Accuracy:  0.250


100%|██████████████████████████████████████████████████████████████████| 33/33 [00:51<00:00,  1.55s/it]


Epochs: 2 | Train Loss:  0.361 | Train Accuracy:  0.606 | Val Loss:  0.325 | Val Accuracy:  0.625


In [8]:
evaluate(model, df_test)

Test Accuracy:  0.667


In [ ]:
pickle.dump(model,open('../model/bert.pckl','wb'))